In [183]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute

In [184]:
# The message to send
sharedQubits = QuantumRegister(2, 'shared')

encodedMessage = ClassicalRegister(2, 'em')

# The qubits containing the classically encoded message
messageQubits = QuantumRegister(2, 'cm')

#### Encoding Layer

Encodes the message into the bellState

In [194]:
encodeCircuit = QuantumCircuit(encodedMessage, messageQubits, sharedQubits)
encodeCircuit.x(messageQubits[0])

encodeCircuit.h(sharedQubits[0])
encodeCircuit.cx(sharedQubits[0], sharedQubits[1])

# Measurement
encodeCircuit.measure(messageQubits, encodedMessage)

# Encoding
encodeCircuit.cx(messageQubits[1], sharedQubits[0])
encodeCircuit.cz(messageQubits[0], sharedQubits[0])

#### Transfer Layer

In [195]:
receiverQubit = QuantumRegister(1, 'r')

transferLayerCircuit = QuantumCircuit(sharedQubits, receiverQubit)
transferLayerCircuit.swap(sharedQubits[0], receiverQubit[0])

In [196]:
# The receiving registers
receivingRegisters = ClassicalRegister(2, 'mdec')

decodeCircuit = QuantumCircuit(receiverQubit, sharedQubits, receivingRegisters)
decodeCircuit.cx(receiverQubit[0], sharedQubits[1])
decodeCircuit.h(receiverQubit[0])
decodeCircuit.measure(receiverQubit[0], receivingRegisters[0])
decodeCircuit.measure(sharedQubits[1], receivingRegisters[1])

#### Visualization

In [197]:
encodeCircuit.draw()

In [198]:
transferLayerCircuit.draw()

In [199]:
decodeCircuit.draw()

In [200]:
allCircuits = (encodeCircuit+transferLayerCircuit+decodeCircuit)
allCircuits.draw()

In [201]:
backend_sim = BasicAer.get_backend('qasm_simulator')
result = execute(allCircuits, backend_sim).result()
counts = result.get_counts(allCircuits)
print(counts)

{'01 01': 1024}
